

---



In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

### 드라이브 마운트해서 해당 네이버 리뷰데이터 가져오기

In [3]:
df

,rate,text
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ
...,...,...
199995,2,장마라그런가!!! 달지않아요
199996,5,다이슨 케이스 구매했어요 다이슨 슈퍼소닉 드라이기 케이스 구매했어요가격 괜찮고 배송...
199997,5,로드샾에서 사는것보다 세배 저렴하네요 ㅜㅜ 자주이용할께요
199998,5,넘이쁘고 쎄련되보이네요~


In [2]:
df = pd.read_table('/content/drive/MyDrive/corpus-master/corpus-master/sentiment/naver_shopping.txt',header = None) #데이터 불러오기
df.columns = ['rate','text'] #데이터의 칼럼명 지정

df.head()

,rate,text
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다. 바느질이 조금 ...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다. 전...
4,5,민트색상 예뻐요. 옆 손잡이는 거는 용도로도 사용되네요 ㅎㅎ


# 데이터 전처리


In [4]:
# 결측값은 딱히 없다
df.isnull().sum()

rate    0
text    0
dtype: int64

In [6]:
# 중복값도 없다
df.duplicated().sum()

0

### 전처리를 위한 krwordrank다운로드 해서 전처리하기

In [3]:
pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### 전처리를 하는데 위에 리뷰를 보면 ㅎㅎ처럼 특정 은어?? 같은 것들을 제거하는 역할임

In [4]:
from krwordrank.hangle import normalize

df['text'] = df['text'].apply(lambda x : normalize(x,english=True,number=True))
df.head()

,rate,text
0,5,배공빠르고 굿
1,2,택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고
2,5,아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉...
3,2,선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화...
4,5,민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요


# konlp로 감성 분석 코드

In [7]:
pip install konlpy #다운로드하기

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 679 kB/s 
     |████████████████████████████████| 465 kB 76.3 MB/s 


In [8]:
# 먼저 학습과 시험데이터로 split하기
# Okt()클래스 생성하기

from sklearn.model_selection import train_test_split
from konlpy.tag import Okt


dataset_train, dataset_test = train_test_split(df, test_size=0.2, shuffle=True, random_state=34)
okt = Okt()

In [ ]:
dataset_train.head()

,rate,text
50608,2,베개유목민인데 복구력이 넘좋다해서 삿지만 너무좋아서 높게느껴질 정도에요 평소 낮은베...
76017,2,역시 저렴한 이유가 있네요 가격에 비해 너무 많은걸 바랬나봐요 딱 가격값하는 엄청 ...
152093,2,제품은 만족하는데 배송이 한진택배라 엄청 오래걸렸어요
95204,1,이건 뭐 쓰던거 파는건가요 뚜껑이 새것처럼 잠금장치가 없네요 물건은 다 샛는지 터졋...
181587,1,신은지 3주만에 밑창 떨어져서 구두방에 만원주고 수선했는데 어차피 금방 또 떨어진다...


### 라벨은 1,2,4,5로 평점으로 5가 제일 좋은 평점임을 의미한다

In [ ]:
#df['rate'].unique() #라벨데이터 종류 확인

In [9]:
# 학습에 쓰기위해서 데이터를 리스트 형태로 바꿔주기

dataset_train = dataset_train.values.tolist()
dataset_test = dataset_test.values.tolist()

In [ ]:
dataset_train[:2]

[[2,
  '베개유목민인데 복구력이 넘좋다해서 삿지만 너무좋아서 높게느껴질 정도에요 평소 낮은베개를 선호해서 자고일어나면 어깨나 목이아프고 푹 잠을 못잔느낌'],
 [2, '역시 저렴한 이유가 있네요 가격에 비해 너무 많은걸 바랬나봐요 딱 가격값하는 엄청 얇은 재질의 상품이에요']]

### 아래코드는 토큰화해서 하나의 파일로 만드는 코드인데 이게 시간이 걸려서 만들어서 이미 파일을 만들어서 마운트하였다면 바로 그냥 실행됨

### 제가 올린 test_docs.json과 train_docs.json을 세션장소에 업로드를 하면 바로 실행됩니다!!

In [5]:
import json
import os
from pprint import pprint

# 리뷰를 토큰화하기 이때 토큰화하면서 품사도 정해진다.
# 이때 기존에 올린 test_docs와 train_docs를 세션장소에 업로드 하면 바로 실행됨 (안하면 엄청 오래걸림)

def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

if os.path.isfile('train_docs.json'):
    with open('train_docs.json') as f:
        train_docs = json.load(f)
    with open('test_docs.json') as f:
        test_docs = json.load(f)
else:
    train_docs = [(tokenize(row[1]), row[0]) for row in dataset_train]
    test_docs = [(tokenize(row[1]), row[0]) for row in dataset_test]
    # JSON 파일로 저장
    with open('train_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(train_docs, make_file, ensure_ascii=False, indent="\t")
    with open('test_docs.json', 'w', encoding="utf-8") as make_file:
        json.dump(test_docs, make_file, ensure_ascii=False, indent="\t")

pprint(train_docs[0])

[['베개/Noun',
  '유목민/Noun',
  '인데/Josa',
  '복구/Noun',
  '력/Suffix',
  '이/Josa',
  '넘다/Verb',
  '좋다/Adjective',
  '다해/Noun',
  '서/Josa',
  '삿/Noun',
  '지만/Josa',
  '너무/Adverb',
  '좋다/Adjective',
  '높다/Adjective',
  '느껴지다/Verb',
  '정도/Noun',
  '에요/Josa',
  '평소/Noun',
  '낮다/Adjective',
  '베개/Noun',
  '를/Josa',
  '선호/Noun',
  '하다/Verb',
  '자고/Noun',
  '일어나다/Verb',
  '어깨/Noun',
  '나/Josa',
  '목/Noun',
  '이/Josa',
  '아프다/Adjective',
  '푹/Noun',
  '잠/Noun',
  '을/Josa',
  '못자다/Verb',
  '느낌/Noun'],
 2]


In [14]:
#train데이터의 토큰 개수 확인하기

tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

2266473


In [15]:
import nltk
text = nltk.Text(tokens, name='NMSC')

# 전체 토큰의 개수
print(len(text.tokens))

# 중복을 제외한 토큰의 개수
print(len(set(text.tokens)))            

# 출현 빈도가 높은 상위 토큰 10개
pprint(text.vocab().most_common(10))

2266473
38147
[('하다/Verb', 78131),
 ('이/Josa', 54095),
 ('좋다/Adjective', 48787),
 ('도/Josa', 39088),
 ('에/Josa', 38327),
 ('가/Josa', 37461),
 ('너무/Adverb', 23529),
 ('배송/Noun', 22801),
 ('은/Josa', 20095),
 ('있다/Adjective', 17928)]


### 가장 많이 사용된 토큰 10000개만 사용해서 분석에 사용하기
이때 시간이 많이 걸렸습니다


In [ ]:
# 10000개하였을떄 30분이상걸림 소모ram 17
# 이 코드가 가장 오래 걸리니 이코드는 피클링된 모델을 쓰니 실행 안해도 됩니다

selected_words = [f[0] for f in text.vocab().most_common(10000)]

def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

In [ ]:
#float32로 바꿔주기
#여기서도 ram많이 잡아먹음 ram 23
#이코드도 실행 안해도 됩니다

import numpy as np

x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')

y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')

In [ ]:
#라벨데이터의 경우 원핫인코딩을 해주기
#이코드도 실행 안해도 됩니다
from keras.utils import np_utils


Y_encoded = np_utils.to_categorical(y_train)
Y_encoded_test = np_utils.to_categorical(y_test)

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics

#이코드도 실행 안해도 됩니다


model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(6, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
             loss=losses.binary_crossentropy,
             metrics=[metrics.binary_accuracy])

model.fit(x_train, Y_encoded, epochs=10, batch_size=512)
results = model.evaluate(x_test, Y_encoded_test)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/rmsprop.py:135: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
313/313 [==============================] - 16s 39ms/step - loss: 0.2767 - binary_accuracy: 0.8823
Epoch 2/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2290 - binary_accuracy: 0.8964
Epoch 3/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2218 - binary_accuracy: 0.9000
Epoch 4/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2160 - binary_accuracy: 0.9033
Epoch 5/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2102 - binary_accuracy: 0.9069
Epoch 6/10
313/313 [==============================] - 3s 8ms/step - loss: 0.2042 - binary_accuracy: 0.9108
Epoch 7/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1980 - binary_accuracy: 0.9147
Epoch 8/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1916 - binary_accuracy: 0.9181
Epoch 9/10
313/313 [==============================] - 3s 8ms/step - loss: 0.1849 - binary_accuracy: 0.9219
Epoch 10/10
1250/1250 [============

In [ ]:
#이코드도 실행 안해도 됩니다
results = model.evaluate(x_test, Y_encoded_test)

1250/1250 [==============================] - 3s 2ms/step - loss: 0.2667 - binary_accuracy: 0.8886


### 테스트 하는 함수 생성하기

In [10]:
#리뷰를 입력하면 리뷰의 평점을 예측하는 함수작성

def predict_pos_neg(model,review):
    token = tokenize(review)
    tf = term_frequency(token)
    data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
    
    b = np.argmax(model.predict(data))


    return b

In [ ]:
predict_pos_neg(model,'감사합니다')

1/1 [==============================] - 0s 74ms/step


5

In [ ]:
predict_pos_neg(model,'아니 성능도 별로고 퀄리티도 안좋아요')

1/1 [==============================] - 0s 19ms/step


1

피클링을통해서 모델 저장

In [ ]:
import pickle
import joblib

#dumps를 통해서 잘 저장됨
saved_model = pickle.dumps(model)

In [ ]:
joblib.dump(model, '감성분석.pkl') 

['감성분석.pkl']

### 피클링된 모델을 여기서 불러와서 테스트하면됩니다
이때 토큰나이저가 있어야함
위에서 함수정의는 실행되어야 합니다!!

위에서 시간이 오래걸린다고 한거는 실행안해도 됨 (단 위에서부터 나머지는 실행을 해야합니다)

In [11]:
#다시 피클링된거를 가져오기
import joblib

test_model = joblib.load('감성분석.pkl') 

In [16]:
#얘는 위의 tokens와 text가 선언되야 합니다 (위쪽에 코드있어요)

selected_words = [f[0] for f in text.vocab().most_common(10000)]

In [17]:
def term_frequency(doc):
    return [doc.count(word) for word in selected_words]

In [18]:
print(predict_pos_neg(test_model,'튼튼하고 좋아요!!')) # 잘 되는 것을 확인!!

1/1 [==============================] - 2s 2s/step
5


In [19]:
print(predict_pos_neg(test_model,'좋아요 만족합니다 헝 만족합니다 튼튼하고 포장도 좋아요 튼튼하게 와요 아쉬운 점은')) # 잘 되는 것을 확인!!

1/1 [==============================] - 0s 14ms/step
5


# lstm모델로 리뷰 생성하기

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
import pandas as pd
import numpy as np
from string import punctuation

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
#위의 리뷰데이터를 사용
#이 모델의 한계점 데이터는 많은데 이걸 일일히 다 인코딩을 하게되니 엄청나게 모델의 메모리 소모가 커서 5천개만 사용하였습니다 ㅠㅠ
#메모리의 한계를 해결하는 것도 해결과제 ㅠㅠ

df_half = df.iloc[:5000,:]

In [ ]:
# 리뷰 값들을 리스트로 저장

headline = []
headline.extend(list(df_half.text.values)) 
headline[:5]

['배공빠르고 굿',
 '택배가 엉망이네용 저희집 밑에층에 말도없이 놔두고가고',
 '아주좋아요 바지 정말 좋아서2개 더 구매했어요 이가격에 대박입니다 바느질이 조금 엉성하긴 하지만 편하고 가성비 최고예요',
 '선물용으로 빨리 받아서 전달했어야 하는 상품이었는데 머그컵만 와서 당황했습니다 전화했더니 바로주신다했지만 배송도 누락되어있었네요 확인안하고 바로 선물했으면 큰일날뻔했네요 이렇게 배송이 오래걸렸으면 사는거 다시 생각했을거같아요 아쉽네요',
 '민트색상 예뻐요 옆 손잡이는 거는 용도로도 사용되네요']

In [ ]:
print('총 샘플의 개수 : {}'.format(len(headline)))

총 샘플의 개수 : 5000


In [ ]:
#토큰 크기

tokenizer = Tokenizer()
tokenizer.fit_on_texts(headline)
vocab_size = len(tokenizer.word_index) + 1
print('단어 집합의 크기 : %d' % vocab_size)

단어 집합의 크기 : 21120


In [ ]:
sequences = list()

for sentence in headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[4323, 186],
 [772, 4324],
 [772, 4324, 4325],
 [772, 4324, 4325, 4326],
 [772, 4324, 4325, 4326, 2346],
 [772, 4324, 4325, 4326, 2346, 4327],
 [1201, 654],
 [1201, 654, 13],
 [1201, 654, 13, 4328],
 [1201, 654, 13, 4328, 8],
 [1201, 654, 13, 4328, 8, 94]]

In [ ]:
index_to_word = {}
for key, value in tokenizer.word_index.items(): # 인덱스를 단어로 바꾸기 위해 index_to_word를 생성
    index_to_word[value] = key

print('빈도수 상위 582번 단어 : {}'.format(index_to_word[582]))

빈도수 상위 582번 단어 : 없어


In [ ]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 36


In [ ]:
#max_len로 패딩처리를 해주기

sequences = pad_sequences(sequences, maxlen=max_len, padding='pre')
print(sequences[:3])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0 4323  186]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0  772 4324]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0  772 4324 4325]]


In [ ]:
#배열형태로 바꾸고 x와 y를 분리하기

sequences = np.array(sequences)
X = sequences[:,:-1]
y = sequences[:,-1]

In [ ]:
#여기서 메모리 용량이 엄청나게 소모된다
#vocab_size는 위에서 구한 토큰의 크기

y = to_categorical(y, num_classes=vocab_size)

In [ ]:
X.shape

(38706, 35)

In [ ]:
y.shape

(38706, 21120)

### 딥러닝 모델 학습하기

이것도 오래걸리니 이걸하고 피클링을 진행해서 다음에는 시간소모를 아끼자

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, LSTM


embedding_dim = 10
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(hidden_units))
model.add(Dense(vocab_size, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=2)

Epoch 1/200
1210/1210 - 26s - loss: 9.6769 - accuracy: 0.0122 - 26s/epoch - 21ms/step
Epoch 2/200
1210/1210 - 11s - loss: 9.0935 - accuracy: 0.0123 - 11s/epoch - 9ms/step
Epoch 3/200
1210/1210 - 11s - loss: 8.7943 - accuracy: 0.0132 - 11s/epoch - 9ms/step
Epoch 4/200
1210/1210 - 11s - loss: 8.4589 - accuracy: 0.0151 - 11s/epoch - 9ms/step
Epoch 5/200
1210/1210 - 11s - loss: 8.0272 - accuracy: 0.0177 - 11s/epoch - 9ms/step
Epoch 6/200
1210/1210 - 11s - loss: 7.5355 - accuracy: 0.0198 - 11s/epoch - 9ms/step
Epoch 7/200
1210/1210 - 11s - loss: 7.0210 - accuracy: 0.0244 - 11s/epoch - 9ms/step
Epoch 8/200
1210/1210 - 11s - loss: 6.5182 - accuracy: 0.0342 - 11s/epoch - 9ms/step
Epoch 9/200
1210/1210 - 11s - loss: 6.0367 - accuracy: 0.0560 - 11s/epoch - 9ms/step
Epoch 10/200
1210/1210 - 11s - loss: 5.5853 - accuracy: 0.0965 - 11s/epoch - 9ms/step
Epoch 11/200
1210/1210 - 11s - loss: 5.1656 - accuracy: 0.1525 - 11s/epoch - 9ms/step
Epoch 12/200
1210/1210 - 11s - loss: 4.7695 - accuracy: 0.2163

### 생성된 모델을 바탕으로 단어를 입력하면 문장을 만들어내는 함수 구현하기

In [ ]:
def sentence_generation(model, tokenizer, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word
    sentence = ''

    # n번 반복
    for _ in range(n):
        encoded = tokenizer.texts_to_sequences([current_word])[0]
        encoded = pad_sequences([encoded], maxlen=max_len-1, padding='pre')

        # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        result = model.predict(encoded, verbose=0)
        result = np.argmax(result, axis=1)

        for word, index in tokenizer.word_index.items(): 
            # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
            if index == result:
                break

        # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        current_word = current_word + ' '  + word

        # 예측 단어를 문장에 저장
        sentence = sentence + ' ' + word

    sentence = init_word + sentence
    return sentence

테스트 해보기

In [ ]:
print(sentence_generation(model, tokenizer, '별로', 10))

별로 잘 지지해 주지는 못하는거 같아요 믿고 하면서 뒤로 값어치할줄알았는데 예쁘고


In [ ]:
print(sentence_generation(model, tokenizer, '좋아요',10 ))

좋아요 만족합니다 헝 만족합니다 튼튼하고 포장도 좋아요 튼튼하게 와요 아쉬운 점은


In [ ]:
#dumps를 통해서 잘 저장됨
saved_model = pickle.dumps(model)

In [ ]:
joblib.dump(model, '문장생성.pkl') 

['문장생성.pkl']

### 이부분에서 피클링된것을 불러와서 테스트하면 됩니다!!

In [ ]:
test_model = joblib.load('문장생성.pkl') 

In [ ]:
sequences = list()

for sentence in headline:

    # 각 샘플에 대한 정수 인코딩
    encoded = tokenizer.texts_to_sequences([sentence])[0] 
    for i in range(1, len(encoded)):
        sequence = encoded[:i+1]
        sequences.append(sequence)

sequences[:11]

[[4323, 186],
 [772, 4324],
 [772, 4324, 4325],
 [772, 4324, 4325, 4326],
 [772, 4324, 4325, 4326, 2346],
 [772, 4324, 4325, 4326, 2346, 4327],
 [1201, 654],
 [1201, 654, 13],
 [1201, 654, 13, 4328],
 [1201, 654, 13, 4328, 8],
 [1201, 654, 13, 4328, 8, 94]]

In [ ]:
max_len = max(len(l) for l in sequences)
print('샘플의 최대 길이 : {}'.format(max_len))

샘플의 최대 길이 : 36


In [ ]:
print(sentence_generation(test_model, tokenizer, '맜있어요', 20))

맜있어요 우알롱에서 다양한 사이즈 너무 너무 배송도 좋아요 너무 좋아요 용량이 넘 힘들어요 여기서 디자인 색깔도 훨씬 만족스럽습니다 것 듭니다 조금
